In [1]:
import pygame
import random
import sys

# Simple Flappy Bird clone with procedural graphics (no external files)
# Paste this into a file named flappy_bird_pygame.py and run:
#    python -m pip install pygame
#    python flappy_bird_pygame.py

WIDTH, HEIGHT = 400, 600
FPS = 60

# Bird properties
BIRD_X = 80
BIRD_RADIUS = 16
GRAVITY = 0.45
FLAP_STRENGTH = -9

# Pipe properties
PIPE_WIDTH = 70
PIPE_GAP = 150
PIPE_SPEED = 3
PIPE_INTERVAL = 1500  # milliseconds

GROUND_HEIGHT = 100

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 36)
small_font = pygame.font.SysFont(None, 24)


def draw_background(surface, t):
    # simple sky gradient that cycles slightly with time
    for i in range(HEIGHT):
        a = i / HEIGHT
        r = int(135 + 40 * a)
        g = int(206 + 20 * a)
        b = int(235 + 10 * a)
        surface.fill((r, g, b), rect=pygame.Rect(0, i, WIDTH, 1))
    # sun
    sun_x = WIDTH - 60
    sun_y = 70
    pygame.draw.circle(surface, (255, 220, 90), (sun_x, sun_y), 30)


class Bird:
    def __init__(self):
        self.x = BIRD_X
        self.y = HEIGHT // 2
        self.vel = 0.0
        self.radius = BIRD_RADIUS
        self.alive = True

    def flap(self):
        self.vel = FLAP_STRENGTH

    def update(self):
        self.vel += GRAVITY
        self.y += self.vel
        # floor collision
        if self.y + self.radius >= HEIGHT - GROUND_HEIGHT:
            self.y = HEIGHT - GROUND_HEIGHT - self.radius
            self.alive = False
        # ceiling collision
        if self.y - self.radius <= 0:
            self.y = self.radius
            self.vel = 0

    def draw(self, surface):
        # body
        pygame.draw.circle(surface, (255, 220, 0), (int(self.x), int(self.y)), self.radius)
        # wing
        wing_offset = 6 * (1 if self.vel < 0 else -1)
        pygame.draw.polygon(surface, (220, 180, 0), [(self.x - 6, self.y), (self.x - 14, self.y + wing_offset), (self.x, self.y + 6)])
        # eye
        pygame.draw.circle(surface, (255, 255, 255), (int(self.x + 6), int(self.y - 6)), 5)
        pygame.draw.circle(surface, (0, 0, 0), (int(self.x + 7), int(self.y - 6)), 2)

    def get_rect(self):
        return pygame.Rect(self.x - self.radius, self.y - self.radius, self.radius * 2, self.radius * 2)


class Pipe:
    def __init__(self, x):
        self.x = x
        self.w = PIPE_WIDTH
        self.gap = PIPE_GAP
        # choose gap position
        self.top = random.randint(60, HEIGHT - GROUND_HEIGHT - 60 - self.gap)
        self.passed = False

    def update(self):
        self.x -= PIPE_SPEED

    def draw(self, surface):
        # top pipe
        pygame.draw.rect(surface, (30, 160, 30), pygame.Rect(self.x, 0, self.w, self.top))
        # bottom pipe
        bottom_y = self.top + self.gap
        pygame.draw.rect(surface, (30, 160, 30), pygame.Rect(self.x, bottom_y, self.w, HEIGHT - GROUND_HEIGHT - bottom_y))
        # pipe rim shading
        pygame.draw.rect(surface, (20, 120, 20), pygame.Rect(self.x - 4, 0, 6, self.top))
        pygame.draw.rect(surface, (20, 120, 20), pygame.Rect(self.x - 4, bottom_y, 6, HEIGHT - GROUND_HEIGHT - bottom_y))

    def collides_with(self, rect: pygame.Rect):
        top_rect = pygame.Rect(self.x, 0, self.w, self.top)
        bottom_rect = pygame.Rect(self.x, self.top + self.gap, self.w, HEIGHT - GROUND_HEIGHT - (self.top + self.gap))
        return rect.colliderect(top_rect) or rect.colliderect(bottom_rect)


def draw_ground(surface):
    pygame.draw.rect(surface, (200, 180, 120), pygame.Rect(0, HEIGHT - GROUND_HEIGHT, WIDTH, GROUND_HEIGHT))
    # simple grass
    for i in range(0, WIDTH, 10):
        pygame.draw.polygon(surface, (50, 140, 50), [(i, HEIGHT - GROUND_HEIGHT), (i + 5, HEIGHT - GROUND_HEIGHT - 12), (i + 10, HEIGHT - GROUND_HEIGHT)])


def main():
    bird = Bird()
    pipes = []
    score = 0
    running = True
    last_pipe_time = pygame.time.get_ticks() - PIPE_INTERVAL
    paused = False
    game_over = False

    while running:
        dt = clock.tick(FPS)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    if not game_over:
                        bird.flap()
                    else:
                        # restart
                        bird = Bird()
                        pipes = []
                        score = 0
                        game_over = False
                if event.key == pygame.K_r and game_over:
                    bird = Bird()
                    pipes = []
                    score = 0
                    game_over = False
                if event.key == pygame.K_ESCAPE:
                    running = False

        if not game_over:
            # spawn pipes
            now = pygame.time.get_ticks()
            if now - last_pipe_time >= PIPE_INTERVAL:
                last_pipe_time = now
                pipes.append(Pipe(WIDTH + 20))

            # update bird
            bird.update()

            # update pipes
            for p in pipes:
                p.update()
                # check passing for score
                if not p.passed and p.x + p.w < bird.x:
                    p.passed = True
                    score += 1

            # remove off-screen pipes
            pipes = [p for p in pipes if p.x + p.w > -50]

            # collisions
            bird_rect = bird.get_rect()
            for p in pipes:
                if p.collides_with(bird_rect):
                    bird.alive = False
                    break

            if not bird.alive:
                game_over = True

        # draw
        draw_background(screen, pygame.time.get_ticks() / 1000.0)
        for p in pipes:
            p.draw(screen)
        draw_ground(screen)
        bird.draw(screen)

        # HUD
        score_surf = font.render(str(score), True, (255, 255, 255))
        score_bg = pygame.Surface((score_surf.get_width() + 12, score_surf.get_height() + 8), pygame.SRCALPHA)
        score_bg.fill((0, 0, 0, 120))
        screen.blit(score_bg, (10, 10))
        screen.blit(score_surf, (16, 14))

        if game_over:
            go_text = font.render("Game Over", True, (255, 50, 50))
            instr = small_font.render("Press SPACE or R to restart, ESC to quit", True, (255, 255, 255))
            screen.blit(go_text, (WIDTH // 2 - go_text.get_width() // 2, HEIGHT // 2 - 40))
            screen.blit(instr, (WIDTH // 2 - instr.get_width() // 2, HEIGHT // 2 + 10))

        pygame.display.flip()

    pygame.quit()
    sys.exit()


if __name__ == '__main__':
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\AL REHMAN LAPTOPS\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import pygame
import sys
import random

# =====================
# 1. SNAKE GAME
# =====================
def snake_game():
    pygame.init()
    WIDTH, HEIGHT = 600, 400
    CELL = 20
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    clock = pygame.time.Clock()
    font = pygame.font.SysFont(None, 36)

    snake = [(100, 100), (80, 100), (60, 100)]
    direction = (CELL, 0)
    food = (random.randrange(0, WIDTH, CELL), random.randrange(0, HEIGHT, CELL))
    score = 0
    running = True

    while running:
        clock.tick(10)
        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            elif e.type == pygame.KEYDOWN:
                if e.key == pygame.K_UP and direction != (0, CELL): direction = (0, -CELL)
                if e.key == pygame.K_DOWN and direction != (0, -CELL): direction = (0, CELL)
                if e.key == pygame.K_LEFT and direction != (CELL, 0): direction = (-CELL, 0)
                if e.key == pygame.K_RIGHT and direction != (-CELL, 0): direction = (CELL, 0)

        head = (snake[0][0] + direction[0], snake[0][1] + direction[1])
        snake.insert(0, head)

        if head == food:
            score += 1
            food = (random.randrange(0, WIDTH, CELL), random.randrange(0, HEIGHT, CELL))
        else:
            snake.pop()

        if (head[0] < 0 or head[0] >= WIDTH or head[1] < 0 or head[1] >= HEIGHT or head in snake[1:]):
            return  # game over

        screen.fill((0,0,0))
        for s in snake:
            pygame.draw.rect(screen, (0,255,0), (*s, CELL, CELL))
        pygame.draw.rect(screen, (255,0,0), (*food, CELL, CELL))
        screen.blit(font.render(f"Score: {score}", True, (255,255,255)), (10, 10))
        pygame.display.flip()

# =====================
# 2. BREAKOUT GAME
# =====================
def breakout_game():
    pygame.init()
    WIDTH, HEIGHT = 500, 400
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    clock = pygame.time.Clock()
    font = pygame.font.SysFont(None, 30)

    paddle = pygame.Rect(WIDTH//2 - 40, HEIGHT - 20, 80, 10)
    ball = pygame.Rect(WIDTH//2, HEIGHT//2, 10, 10)
    ball_vel = [4, -4]
    bricks = [pygame.Rect(x*50+5, y*20+5, 40, 15) for x in range(9) for y in range(5)]

    running = True
    score = 0
    while running:
        clock.tick(60)
        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                pygame.quit(); sys.exit()

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and paddle.left > 0:
            paddle.move_ip(-6, 0)
        if keys[pygame.K_RIGHT] and paddle.right < WIDTH:
            paddle.move_ip(6, 0)

        ball.move_ip(ball_vel)
        if ball.left <= 0 or ball.right >= WIDTH:
            ball_vel[0] = -ball_vel[0]
        if ball.top <= 0:
            ball_vel[1] = -ball_vel[1]
        if ball.colliderect(paddle):
            ball_vel[1] = -ball_vel[1]
        for b in bricks[:]:
            if ball.colliderect(b):
                bricks.remove(b)
                ball_vel[1] = -ball_vel[1]
                score += 10
                break
        if ball.bottom >= HEIGHT:
            return  # game over

        screen.fill((0,0,20))
        pygame.draw.rect(screen, (200,200,200), paddle)
        pygame.draw.ellipse(screen, (200,50,50), ball)
        for b in bricks:
            pygame.draw.rect(screen, (50,200,50), b)
        screen.blit(font.render(f"Score: {score}", True, (255,255,0)), (10,10))
        pygame.display.flip()

# =====================
# 3. CAR DODGING GAME
# =====================
def car_game():
    pygame.init()
    WIDTH, HEIGHT = 400, 600
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    clock = pygame.time.Clock()
    font = pygame.font.SysFont(None, 36)

    car = pygame.Rect(WIDTH//2 - 20, HEIGHT - 80, 40, 60)
    enemies = []
    enemy_timer = 0
    score = 0

    running = True
    while running:
        dt = clock.tick(60)
        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                pygame.quit(); sys.exit()

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and car.left > 0:
            car.move_ip(-5,0)
        if keys[pygame.K_RIGHT] and car.right < WIDTH:
            car.move_ip(5,0)

        enemy_timer += dt
        if enemy_timer > 1000:
            enemy_timer = 0
            enemies.append(pygame.Rect(random.randint(0, WIDTH-40), -60, 40, 60))

        for en in enemies:
            en.move_ip(0,5)
        enemies = [en for en in enemies if en.top < HEIGHT]

        for en in enemies:
            if car.colliderect(en):
                return  # game over

        score += 1

        screen.fill((50,50,50))
        pygame.draw.rect(screen, (0,0,255), car)
        for en in enemies:
            pygame.draw.rect(screen, (255,0,0), en)
        screen.blit(font.render(f"Score: {score}", True, (255,255,255)), (10,10))
        pygame.display.flip()

# =====================
# MAIN MENU
# =====================
def main():
    while True:
        choice = input("Choose game: 1=Snake, 2=Breakout, 3=Car Dodging, q=Quit: ")
        if choice == '1':
            snake_game()
        elif choice == '2':
            breakout_game()
        elif choice == '3':
            car_game()
        elif choice.lower() == 'q':
            break

if __name__ == '__main__':
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 